In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Oct_dict_JSON.json', 'r') as json_file:
    link_day_minute_Oct_dict_JSON = json.load(json_file)

In [11]:
weekend_Oct_list = [6, 7, 13, 14, 20, 21, 27, 28]
#weekend_Oct_list = [7, 14, 21, 28]

In [12]:
link_day_minute_Oct_list = []
for link_idx in range(24):
    for day in weekend_Oct_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Oct_list.append(link_day_minute_Oct_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [13]:
len(link_day_minute_Oct_list)

23040

In [14]:
x = np.matrix(link_day_minute_Oct_list)
x = np.matrix.reshape(x, 24, 960)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [15]:
x[:,:2]

matrix([[ 1636.19355318,  1417.12543889],
        [ 1626.6008501 ,   932.620851  ],
        [ 3824.94048709,  2290.57694614],
        [ 3275.51449472,  3251.6479074 ],
        [ 2840.15499877,  2727.73950219],
        [  869.65389884,  1725.22850698],
        [ 1302.00150238,   824.48731805],
        [ 1061.44794093,  1236.70943259],
        [ 2129.76838186,  1860.10517061],
        [ 1663.97337137,  2059.62699908],
        [ 4877.42602393,  5558.14668839],
        [ 2798.93966838,  2950.75824635],
        [  882.16639739,  1065.60229705],
        [ 1309.41071724,  1641.84976528],
        [ 1356.6688995 ,  2034.53837987],
        [ 2165.83854997,  1816.72588542],
        [  730.02499282,   923.18088498],
        [ 1466.53754782,  2091.50284544],
        [ 2653.85126334,  1246.48446434],
        [ 2355.96909926,  2097.48561538],
        [ 4766.85888951,  4098.48344455],
        [ 1317.41529853,  1012.97239054],
        [ 2578.56685091,  2058.14904278],
        [ 1126.84459114,  1658.845

In [16]:
np.size(x, 0), np.size(x, 1)

(24, 958)

In [17]:
np.size(A,0), np.size(A,1)

(24, 140)

In [18]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [3e-02, 8e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.84832224e+06  0.00000000e+00  6.09e+03 1.42e+02  9.48e+05     0s
   1   1.27322254e+06 -1.94205757e+06  1.06e+03 1.63e+02  2.25e+05     0s
   2   1.08848697e+06 -1.42132691e+06  1.61e+02 2.55e+01  6.72e+04     0s
   3   1.93969537e+05 -3.41917396e+05  1.61e-04 1.14e-13  9.5

In [19]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Oct_weekend.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [20]:
lam_list

[1.1282414000052355e-09,
 3238.871645054255,
 1.2613793157140417e-09,
 305.11984195773414,
 1104.8076937279284,
 3.440344286610693e-06,
 3.428285832902002e-07,
 0.002240437986217543,
 3.4638412925565238e-09,
 2317.1904218082213,
 3.4638412914890736e-09,
 3.463841292439246e-09,
 3.460357301615304e-09,
 3.4802005365780836e-09,
 2411.8440690065895,
 127.03825567948354,
 22.724111905582674,
 282.8393045952599,
 1578.2618067726537,
 2780.8128794791396,
 3.441361829883031e-06,
 750.0271144075888,
 1398.2742372280845,
 467.23090574676286,
 282.057506270594,
 936.3517246504084,
 1.9724209089572963e-06,
 2106.3541048765182,
 0.5868134164704857,
 0.5613871803353043,
 649.7653042253831,
 1333.5664813781436,
 438.50024420528956,
 1.952650430377177e-06,
 1.953642110702689e-06,
 1630.1479395341416,
 198.8838021736137,
 0.00015993629722903138,
 2.8514444639117755,
 1335.8497993189108,
 705.2212139975547,
 491.1948057024839,
 4.5357518035030135e-06,
 4.535749266152381e-06,
 1990.4409853467905,
 2.3049